# Neutryx Pricing Demo

This notebook demonstrates pricing capabilities of the Neutryx library using Python bindings.

## Features Demonstrated
- Vanilla option pricing (Black-Scholes)
- Forward pricing
- Interest rate swap valuation
- Greeks calculation

In [ ]:
# Import Neutryx Python bindings
# Note: service_python must be built with `maturin develop`
try:
    import neutryx
except ImportError:
    print("Neutryx not installed. Run: cd crates/service_python && maturin develop")
    neutryx = None

import numpy as np
import matplotlib.pyplot as plt

## 1. Black-Scholes Option Pricing

Price a European call option using the Black-Scholes model.

In [ ]:
# Option parameters
spot = 100.0
strike = 105.0
time_to_maturity = 1.0  # years
risk_free_rate = 0.05
volatility = 0.20

# Pure Python Black-Scholes for comparison
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """Black-Scholes call option price."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

bs_price = black_scholes_call(spot, strike, time_to_maturity, risk_free_rate, volatility)
print(f"Black-Scholes Call Price: {bs_price:.4f}")

In [ ]:
# Using Neutryx (when available)
if neutryx:
    option = neutryx.VanillaOption(
        spot=spot,
        strike=strike,
        maturity=time_to_maturity,
        is_call=True,
        volatility=volatility,
        risk_free_rate=risk_free_rate
    )
    neutryx_price = option.price()
    print(f"Neutryx Call Price: {neutryx_price:.4f}")
else:
    print("Neutryx bindings not available")

## 2. Strike vs Price Plot

In [ ]:
# Price options across different strikes
strikes = np.linspace(80, 120, 41)
call_prices = [black_scholes_call(spot, K, time_to_maturity, risk_free_rate, volatility) for K in strikes]

plt.figure(figsize=(10, 6))
plt.plot(strikes, call_prices, 'b-', linewidth=2, label='Call Option')
plt.axvline(x=spot, color='gray', linestyle='--', label='Spot')
plt.xlabel('Strike Price')
plt.ylabel('Option Price')
plt.title('European Call Option Price vs Strike')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Greeks Calculation

Calculate Delta, Gamma, Vega, and Theta.

In [ ]:
def black_scholes_greeks(S, K, T, r, sigma):
    """Calculate Black-Scholes Greeks."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100  # per 1% vol
    theta = -(S*norm.pdf(d1)*sigma)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2)
    theta = theta / 365  # per day
    
    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta}

greeks = black_scholes_greeks(spot, strike, time_to_maturity, risk_free_rate, volatility)

print("Greeks for ATM Call Option:")
print(f"  Delta: {greeks['delta']:.4f}")
print(f"  Gamma: {greeks['gamma']:.4f}")
print(f"  Vega:  {greeks['vega']:.4f}")
print(f"  Theta: {greeks['theta']:.4f}")

## 4. Forward Pricing

In [ ]:
# Forward contract pricing
spot_price = 100.0
delivery_price = 102.0
time_to_delivery = 0.5  # 6 months
r = 0.05

# Forward value = S - K * exp(-r*T)
forward_value = spot_price - delivery_price * np.exp(-r * time_to_delivery)
print(f"Forward Contract Value: {forward_value:.4f}")

# Fair forward price
fair_forward = spot_price * np.exp(r * time_to_delivery)
print(f"Fair Forward Price: {fair_forward:.4f}")

## Summary

This notebook demonstrated:
1. Vanilla option pricing with Black-Scholes
2. Strike sensitivity analysis
3. Greeks calculation (Delta, Gamma, Vega, Theta)
4. Forward contract valuation

For full Neutryx integration, build the Python bindings with:
```bash
cd crates/service_python
maturin develop
```